In [ ]:
# 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

import lightgbm as lgb
import joblib

# 데이터 로드
train = pd.read_csv('data/processed/cleaned_data/train_clean.csv')
test = pd.read_csv('data/processed/cleaned_data/test_clean.csv')

# EDA: target 분포 확인
plt.figure(figsize=(8,6))
sns.histplot(train['target'], bins=50, kde=True)
plt.title('Target 분포')
plt.xlabel('target')
plt.ylabel('Count')
plt.show()

# EDA: 주요 피처 분포 확인
plt.figure(figsize=(8,6))
sns.boxplot(x=train['전용면적'])
plt.title('전용면적 분포')
plt.show()

# EDA: 상관관계 히트맵
corr = train.corr()
plt.figure(figsize=(12,10))
sns.heatmap(corr, annot=True, fmt=".2f")
plt.title('상관관계 히트맵')
plt.show()

# EDA: 산점도 (전용면적 vs target)
plt.figure(figsize=(8,6))
sns.scatterplot(x='전용면적', y='target', data=train)
plt.title('전용면적 vs Target')
plt.show()

# 파생 변수 생성: 계약일자 -> 년, 월, 분기
train['계약일자'] = pd.to_datetime(train['계약일자'], format='%Y%m%d')
train['계약년'] = train['계약일자'].dt.year
train['계약월'] = train['계약일자'].dt.month
train['계약분기'] = train['계약일자'].dt.quarter

# 파생 변수 생성: 연식 기준 신축여부
train['신축여부'] = train['연식'].apply(lambda x: 1 if x <= 5 else 0)

# 파생 변수 생성: 구 이름 추출
train['구'] = train['자치구'].str.split().str[1]

# test 데이터에도 동일 파생 변수 적용
test['계약일자'] = pd.to_datetime(test['계약일자'], format='%Y%m%d')
test['계약년'] = test['계약일자'].dt.year
test['계약월'] = test['계약일자'].dt.month
test['계약분기'] = test['계약일자'].dt.quarter

test['신축여부'] = test['연식'].apply(lambda x: 1 if x <= 5 else 0)

test['구'] = test['자치구'].str.split().str[1]

# 모델 입력 변수 설정
feature_cols = [
    '전용면적', '연식', '강남3구여부', '구', '계약년', '계약월', '계약분기', '신축여부',
    '반경_1km_지하철역_수', '반경_500m_지하철역_수', '반경_300m_지하철역_수',
    '반경_1km_버스정류장_수', '반경_500m_버스정류장_수', '반경_300m_버스정류장_수',
    '총인구수', '성비(남/여)', 'loanrate_1m', 'loanrate_3m', 'loanrate_6m', 'loanrate_12m'
]
X = train[feature_cols]
y = train['target']

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 전처리 파이프라인 정의
numeric_features = [
    '전용면적', '연식', '강남3구여부', '계약년', '계약월', '계약분기', '신축여부',
    '반경_1km_지하철역_수', '반경_500m_지하철역_수', '반경_300m_지하철역_수',
    '반경_1km_버스정류장_수', '반경_500m_버스정류장_수', '반경_300m_버스정류장_수',
    '총인구수', '성비(남/여)', 'loanrate_1m', 'loanrate_3m', 'loanrate_6m', 'loanrate_12m'
]
categorical_features = ['구']

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('ohe', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# 모델 파이프라인 구성 (전처리 + LightGBM)
model = lgb.LGBMRegressor(random_state=42)
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# 하이퍼파라미터 탐색 설정
param_dist = {
    'model__num_leaves': [31, 50, 70],
    'model__learning_rate': [0.1, 0.01, 0.001],
    'model__max_depth': [-1, 10, 20]
}
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=1,
    random_state=42
)
search.fit(X_train, y_train)

# 최적 모델로 검증 데이터 예측 및 평가
best_model = search.best_estimator_
y_pred = best_model.predict(X_val)

mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
print(f"MAE: {mae:.2f}, R2: {r2:.3f}")

# 잔차 분석: 분포 & 산점도
residuals = y_val - y_pred
plt.figure(figsize=(8,6))
sns.histplot(residuals, kde=True)
plt.title('Residuals Distribution')
plt.show()

plt.figure(figsize=(8,6))
plt.scatter(y_val, y_pred)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--')
plt.title('Actual vs Predicted')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()

# 피처 중요도 시각화
importances = best_model.named_steps['model'].feature_importances_
# OHE 후 컬럼 이름 생성
ohe_features = best_model.named_steps['preprocessor'] \
    .transformers_[1][1] \
    .named_steps['ohe'] \
    .get_feature_names_out(categorical_features)
feature_names = numeric_features + list(ohe_features)
fi = pd.Series(importances, index=feature_names).sort_values()  
plt.figure(figsize=(10,8))
fi.plot.barh()
plt.title('Feature Importances')
plt.show()

# 모델 저장
date_str = pd.Timestamp.now().strftime('%Y%m%d')
joblib.dump(best_model, f'lgbm_price_model_{date_str}.pkl')
